In [1]:
import pandas as pd

In [4]:
schema = pd.read_excel('/content/drive/MyDrive/challenge/Customer Entity Resolution _ Application Prefill/Application Form Schema.xlsx')
credit_data = pd.read_excel('/content/drive/MyDrive/challenge/Customer Entity Resolution _ Application Prefill/Customer Demographics - Credit Bureau.xlsx')
training_id_match = pd.read_excel('/content/drive/MyDrive/challenge/Customer Entity Resolution _ Application Prefill/Training/Training Data - Ground Truth.xlsx')
training_cust_info = pd.read_excel('/content/drive/MyDrive/challenge/Customer Entity Resolution _ Application Prefill/Training/Training Data.xlsx')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
  import nltk
  nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize

In [7]:
def get_score(txt1, txt2):

  # Tokenize the rows
  tokens1 = word_tokenize(txt1)
  tokens2 = word_tokenize(txt2)

  # Convert tokens to lowercase
  tokens1 = [token.lower() for token in tokens1]
  tokens2 = [token.lower() for token in tokens2]

  # Calculate cosine similarity
  vectorizer = CountVectorizer().fit_transform([txt1, txt2])
  vectors = vectorizer.toarray()
  cosine_sim = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1, -1))[0][0]


  return cosine_sim


In [8]:
cust_info_lst = []
credit_info_lst = []
credit_info_id = []
score = []

for i in range(training_cust_info.shape[0]):

  a = credit_data[credit_data['id'] == training_id_match['Credit Bureau ID'][i]]
  b = training_cust_info[training_cust_info['Cust ID'] == training_id_match['Cust ID'][i]]


  selected_values1 = b.iloc[0, 1:]
  single_row_text1 = ' '.join(map(str, selected_values1))
  single_row_text1 = single_row_text1.lower()
  cust_info_lst.append(single_row_text1)

  selected_values2 = a.iloc[0, 1:]
  single_row_text2 = ' '.join(map(str, selected_values2))
  single_row_text2 = single_row_text2.lower()
  credit_info_lst.append(single_row_text2)

  credit_id = a.iloc[0,:1]
  credit_info_id.append(credit_id)


  sim_score = get_score(single_row_text1,single_row_text2)
  score.append(sim_score)



In [9]:
def get_matching_data(txt, lst):

   fixed_sent = txt
   scores_list = []
   for x in lst:
      var_sent = x

      # Tokenize the rows
      tokens1 = word_tokenize(fixed_sent)
      tokens2 = word_tokenize(var_sent)

      # Convert tokens to lowercase
      tokens1 = [token.lower() for token in tokens1]
      tokens2 = [token.lower() for token in tokens2]

      # Calculate cosine similarity
      vectorizer = CountVectorizer().fit_transform([fixed_sent, var_sent])
      vectors = vectorizer.toarray()
      cosine_sim = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1, -1))[0][0]

      scores_list.append(cosine_sim)

   return scores_list


In [10]:
from pandas.io.stata import StataParser

temp = []


# Take inputs Records

first_name = "mateo" # @param {type:"string"}
if first_name == '':
    first_name = 'nan'
temp.append(first_name)
middle_name = "nan" # @param {type:"string"}
if middle_name == '':
    middle_name = 'nan'
temp.append(middle_name)
last_name = "sample" # @param {type:"string"}
if last_name == '':
    last_name = 'nan'
temp.append(last_name)
prefix = '' # @param {type:"string"}
if prefix == '':
    prefix = 'nan'
temp.append(prefix)
suffix = "nan" # @param {type:"string"}
if suffix == '':
    suffix = 'nan'
temp.append(suffix)
address1 = "123 north state"# @param {type:"string"}
if address1 == '':
    address1 = 'nan'
temp.append(address1)
address2 = "st. lansing" # @param {type:"string"}
if address2 == '':
    address2 = 'nan'
temp.append(address2)
city = "nan"  # @param {type:"string"}
if city == '':
    city = 'nan'
temp.append(city)
state = "michigan" # @param {type:"string"}
if state == '':
    state = 'nan'
temp.append(state)
zip = "nan" # @param {type:"string"}
if zip == '':
    zip = 'nan'
temp.append(zip)
dob = "1996-08-10" # @param {type:"string"}
if dob == '':
    dob = 'nan'
dob = dob + ' 00:00:00'
temp.append(dob)

search_info = ' '.join(temp)
print(f'The information entered is: \n {search_info}')

print()
print('Fetched Data is:')
output = get_matching_data(search_info, credit_info_lst)
index = output.index(max(output))
print(credit_info_lst[index])
temp_df = credit_data[credit_data['id'] == credit_info_id[index][0]]
print()

# Convert DataFrame to dictionary
result_dict = temp_df.to_dict(orient='records')[0]
for key, value in result_dict.items():
     print(f"'{key}': {value}")

print()
temp_df.to_csv('Credit_Information.csv')
print('Information Downloaded in excel................')

The information entered is: 
 mateo nan sample nan nan 123 north state st. lansing nan michigan nan 1996-08-10 00:00:00

Fetched Data is:
mateo nan sample nan nan 123 north state st. lansing, mi 48918-0000 nan nan michigan 48918-0000 1996-08-10 00:00:00

'id': 66178
'first_name': Mateo
'Middle name': nan
'last_name': Sample
'prefix': nan
'suffix': nan
'addr 1': 123 North State St. Lansing, MI 48918-0000
'addr 2': nan
'city': nan
'state': Michigan
'zip': 48918-0000
'DOB': 1996-08-10 00:00:00

Information Downloaded in excel................
